In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.6 MB/s eta 0:00:00


In [3]:
import sqlite3
import random
from faker import Faker

# --- Configuration ---
DATABASE_NAME = 'Ecommerce_db.sqlite'   # Output SQLite file
NUM_CUSTOMERS = 1000                                 # Target number of customer rows
NUM_PRODUCTS = 100                                   # Number of product rows
NUM_ORDER_ITEMS = 1500                               # Order items must be >= 1000
MAX_ORDER_ID = 400                                   # Limits number of distinct orders (multiple items per order)
FAKE = Faker()                                       # Used to generate realistic fake data

# ============================================================#
#   1. DATABASE SETUP                                         #
# ============================================================#

def setup_database(db_name):
    """
    Creates a fresh SQLite database, removes old tables,
    and defines a schema with proper keys, constraints,
    and foreign key relationships.
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # SQLite needs this manually enabled or foreign keys won't work
    cursor.execute('PRAGMA foreign_keys = ON;')

    # Drop tables if they exist so the script always produces a clean database
    cursor.execute('DROP TABLE IF EXISTS Order_Items;')
    cursor.execute('DROP TABLE IF EXISTS Customers;')
    cursor.execute('DROP TABLE IF EXISTS Products;')

    # ----------------- PRODUCTS TABLE -----------------
    # Contains product name, category, price, and stock level
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Products (
            product_id INTEGER PRIMARY KEY,
            product_name TEXT NOT NULL UNIQUE,
            category TEXT NOT NULL,
            base_price REAL NOT NULL CHECK (base_price > 0.0),
            stock_level INTEGER NOT NULL CHECK (stock_level >= 0)
        );
    ''')

    # ----------------- CUSTOMERS TABLE -----------------
    # Contains customer info and a UNIQUE email field (used later for testing duplicates)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Customers (
            customer_id INTEGER PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT UNIQUE,
            loyalty_tier TEXT NOT NULL
        );
    ''')

    # ----------------- ORDER ITEMS TABLE -----------------
    # Large table with mixed data types:
    # - Composite Primary Key (order_id, item_number)
    # - References Products & Customers
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Order_Items (
            order_id INTEGER,
            item_number INTEGER,
            product_id INTEGER,
            customer_id INTEGER NOT NULL,
            quantity INTEGER NOT NULL CHECK (quantity >= 1),
            unit_price REAL NOT NULL,
            discount_level TEXT NOT NULL,
            shipping_time_days INTEGER,
            warehouse_location TEXT NOT NULL,

            PRIMARY KEY (order_id, item_number),

            FOREIGN KEY (product_id) REFERENCES Products(product_id),
            FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
        );
    ''')

    conn.commit()
    return conn

# ============================================================#
#   2. PRODUCT GENERATION                                     #
# ============================================================#

def generate_products(conn):
    """
    Creates 100 products across multiple categories.
    Prices vary depending on the category.
    Returns:
      - A list of valid product_ids
      - A price_map of {product_id: base_price}
    """
    cursor = conn.cursor()
    products = []

    product_categories = ['Electronics', 'Home Goods', 'Apparel', 'Books', 'Tools']

    for i in range(1, NUM_PRODUCTS + 1):
        category = random.choice(product_categories)

        # Category affects price ranges
        if category == 'Electronics':
            base_price = round(random.uniform(50.0, 500.0), 2)
        elif category == 'Apparel':
            base_price = round(random.uniform(15.0, 75.0), 2)
        else:
            base_price = round(random.uniform(5.0, 150.0), 2)

        # Fake product name comes from faker
        product = (
            i,
            FAKE.unique.catch_phrase(),
            category,
            base_price,
            random.randint(5, 500)  # stock_level
        )
        products.append(product)

    cursor.executemany('INSERT INTO Products VALUES (?, ?, ?, ?, ?)', products)

    # Retrieve actual IDs and prices back from the DB
    cursor.execute("SELECT product_id, base_price FROM Products;")
    actual_products = cursor.fetchall()

    actual_product_ids = [p[0] for p in actual_products]
    price_map = {p[0]: p[1] for p in actual_products}

    conn.commit()
    print(f"Inserted {len(actual_product_ids)} products.")
    return actual_product_ids, price_map

# ============================================================
#   3. CUSTOMER GENERATION
# ============================================================

def generate_customers(conn):
    """
    Generates customer rows with:
      - Random names + random emails
      - 5% NULL emails (missing data)
      - Intentional duplicate emails (constraint test)
    Uses INSERT OR IGNORE so duplicate emails are skipped.
    Returns:
      - Only IDs of successfully inserted customers
    """
    cursor = conn.cursor()
    customers = []
    loyalty_tiers = ['Bronze', 'Silver', 'Gold']
    DUPLICATE_EMAIL = 'duplicate.test@example.com'  # For intentional constraint failures

    for i in range(1, NUM_CUSTOMERS + 1):
        first_name = FAKE.first_name()
        last_name = FAKE.last_name()
        loyalty_tier = random.choices(loyalty_tiers, weights=[60, 30, 10], k=1)[0]
        email = FAKE.email()

        # 5% of email fields are NULL (missing data realism)
        if random.random() < 0.05:
            email = None
        # Inject the same duplicate email into two rows intentionally
        elif i in [10, 50]:
            email = DUPLICATE_EMAIL

        customers.append((i, first_name, last_name, email, loyalty_tier))

    # Duplicate email constraint will cause row 50 to be ignored
    cursor.executemany('INSERT OR IGNORE INTO Customers VALUES (?, ?, ?, ?, ?)', customers)

    # Retrieve only the valid/included rows
    cursor.execute("SELECT customer_id FROM Customers;")
    actual_customer_ids = [row[0] for row in cursor.fetchall()]

    conn.commit()
    print(f"Inserted {len(actual_customer_ids)} customers (some skipped).")
    return actual_customer_ids

# ============================================================
#   4. ORDER ITEM GENERATION
# ============================================================

def generate_order_items(conn, customer_ids, product_ids, price_map):
    """
    Creates >= 1000 order-item rows.
    Handles composite primary key (order_id + item_number).
    Ensures:
      - Valid customer_id
      - Valid product_id
      - unit_price matches the product base price
      - 2% missing shipping_time_days (NULL)
      - Realistic discount distribution
    """
    cursor = conn.cursor()
    order_items = []

    current_item_counter = {}  # Tracks item_number per order_id

    discount_levels = ['None', 'Small', 'Medium', 'Large']
    warehouse_locations = ['Warehouse A', 'Warehouse B', 'Warehouse C']

    for i in range(1, NUM_ORDER_ITEMS + 1):
        # Pick order_id (many items will share same order)
        order_id = random.randint(1, MAX_ORDER_ID)

        # Increment item_number for that order
        current_item_counter[order_id] = current_item_counter.get(order_id, 0) + 1
        item_number = current_item_counter[order_id]

        # Pick valid foreign keys
        customer_id = random.choice(customer_ids)
        product_id = random.choice(product_ids)

        # Quantity (ratio data)
        quantity = random.randint(1, 5)

        # Unit price = official product price
        unit_price = price_map[product_id]

        # Ordinal discount
        discount_level = random.choices(discount_levels, weights=[70, 20, 7, 3], k=1)[0]

        # 2% NULL interval data for realism
        shipping_time_days = random.randint(1, 14)
        if random.random() < 0.02:
            shipping_time_days = None

        warehouse_location = random.choice(warehouse_locations)

        order_items.append((
            order_id,
            item_number,
            product_id,
            customer_id,
            quantity,
            unit_price,
            discount_level,
            shipping_time_days,
            warehouse_location
        ))

    cursor.executemany('''
        INSERT INTO Order_Items VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', order_items)

    conn.commit()
    print(f"Inserted {NUM_ORDER_ITEMS} order items.")

# ============================================================
#   5. MAIN EXECUTION
# ============================================================

if __name__ == '__main__':
    print(f"--- Starting Database Generation: {DATABASE_NAME} ---")

    # Create tables
    conn = setup_database(DATABASE_NAME)

    # Generate products
    product_ids, price_map = generate_products(conn)

    # Generate customers
    customer_ids = generate_customers(conn)

    # Generate order items
    generate_order_items(conn, customer_ids, product_ids, price_map)

    conn.close()
    print("--- Database generation complete. File saved. ---")


--- Starting Database Generation: Ecommerce_db.sqlite ---
Inserted 100 products.
Inserted 998 customers (some skipped).
Inserted 1500 order items.
--- Database generation complete. File saved. ---
